In [1]:
### Import of packages
%matplotlib notebook

import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from geopy.geocoders import Nominatim
from plotly.subplots import make_subplots
import matplotlib as mpl
import calmap
import calplot
from urllib.request import urlopen
from plotly.offline import init_notebook_mode
import json
import itertools
import plotly.io as pio
import geopandas as gpd
init_notebook_mode(connected=True)
pio.renderers.default = "plotly_mimetype+notebook"

In [2]:
### Import data
#### 'flights.csv' can be downloaded on https://www.kaggle.com/datasets/usdot/flight-delays?select=flights.csv
df_flights = pd.read_csv('flights.csv')
df_airports = pd.read_csv('https://raw.githubusercontent.com/Rody567/TIL6022-LabAssignments/main/Project/Data/airports.csv')
### Make sample data to decrease running times of notebook
Sample_Data = df_flights.sample(n=200000, random_state=1)

C:\Users\hugoo\AppData\Local\Temp\ipykernel_5932\3736060882.py:3: DtypeWarning:

Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
### Create new columns for magnitude of delay, 
### one where delays are made 0 when a flight has a negative delay 
### and one where it is indicated if a delay is bigger than 15 minutes
Sample_Data['DEP_DELAY_POS'] = ''
Sample_Data['DEP_DELAY_over15'] = ''

for i in range(len(Sample_Data)):
    if Sample_Data.DEPARTURE_DELAY.iloc[i] < 0:
        Sample_Data.DEP_DELAY_POS.iloc[i] = 0
        Sample_Data.DEP_DELAY_over15.iloc[i] = 0
    elif Sample_Data.DEPARTURE_DELAY.iloc[i] >= 15:
        Sample_Data.DEP_DELAY_POS.iloc[i] = Sample_Data.DEPARTURE_DELAY.iloc[i]
        Sample_Data.DEP_DELAY_over15.iloc[i] = 1
    else:
        Sample_Data.DEP_DELAY_POS.iloc[i] = Sample_Data.DEPARTURE_DELAY.iloc[i]
        Sample_Data.DEP_DELAY_over15.iloc[i] = 0

C:\Users\hugoo\AppData\Local\Temp\ipykernel_5932\2029083023.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hugoo\AppData\Local\Temp\ipykernel_5932\2029083023.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hugoo\AppData\Local\Temp\ipykernel_5932\2029083023.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hugoo\AppData\Local\Temp\ipykernel_5932\2029083023.py:13: SettingWithCopyWarning:


A value

In [4]:
### Convert day of the week into ordered categorical
def make_categorical(col, categories_ordered):
    variables_ordered = pd.api.types.CategoricalDtype(ordered = True, categories = categories_ordered)
    Sample_Data[col] = Sample_Data[col].astype(variables_ordered)

col = "DAY_OF_WEEK"
categories_ordered = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', "Saturday", 'Sunday']
Sample_Data[col] = Sample_Data[col].astype(str)
day_of_week_dict = {'1': 'Monday', '2' : "Tuesday", '3': 'Wednesday',
               '4': 'Thursday', '5': 'Friday', '6': 'Saturday',
               '7': 'Sunday'}
Sample_Data[col] = Sample_Data[col].replace(day_of_week_dict)
make_categorical(col, categories_ordered)

### Convert DEP_DELAY_over15 to string and rename them to be descriptive
Sample_Data['DEP_DELAY_over15'] = Sample_Data['DEP_DELAY_over15'].astype(str)
delay_dict = {'0.0': '< 15 min', '1.0' : ">= 15 min"}
Sample_Data['DEP_DELAY_over15'] = Sample_Data['DEP_DELAY_over15'].replace(delay_dict)

### Convert CANCELLATION_CODE to the meaning of the cancellation
col = "CANCELLATION_REASON"
cancellation_reason_dict = {'A': 'carrier', 'B' : "weather", 'C': 'NAS',
               'D': 'security'}
Sample_Data[col] = Sample_Data[col].replace(cancellation_reason_dict)

### Convert DEP_TIME and ARR_TIME from string to date
def make_datetime_from_float(col):    
    Sample_Data[col] = pd.to_datetime(Sample_Data[col], format='%H%M', errors = 'coerce').dt.time
    
make_datetime_from_float('DEPARTURE_TIME')
make_datetime_from_float('SCHEDULED_ARRIVAL')

In [5]:
# Exporting DataFrames to import in the main notebook            
Sample_Data.to_pickle("./sample_data")

In [6]:
### Create list of all flights with delay larger than 6 hours
airports = df_flights.ORIGIN_AIRPORT.unique()

origin1 = []
destination1 = []
for i in range(len(df_flights)):
    if df_flights.ARRIVAL_DELAY[i] >= 6*60:
        origin1.append(df_flights.ORIGIN_AIRPORT[i])
        destination1.append(df_flights.DESTINATION_AIRPORT[i])
                                                  
### Create list of all flights with delay larger than 24 hours
origin2 = []
destination2 = []
for i in range(len(df_flights)):
    if df_flights.ARRIVAL_DELAY[i] >= 24*60:
        origin2.append(df_flights.ORIGIN_AIRPORT[i])
        destination2.append(df_flights.DESTINATION_AIRPORT[i])

In [7]:
### Create data frames
df_flights_6hours = pd.DataFrame(index = origin1)
df_flights_6hours["Origin"] = origin1
df_flights_6hours["Destination"] = destination1

df_flights_24hours = pd.DataFrame(index = origin2)
df_flights_24hours["Origin"] = origin2
df_flights_24hours["Destination"] = destination2

In [8]:
### Find coordinates of airport pairs
df_flights_6hours = df_flights_6hours.rename(columns={'Origin': 'IATA1'})
df_flights_6hours = df_flights_6hours.rename(columns={'Destination': 'IATA2'})

df_flights_24hours = df_flights_24hours.rename(columns={'Origin': 'IATA1'})
df_flights_24hours = df_flights_24hours.rename(columns={'Destination': 'IATA2'})

### Add empty columns to the DataFrame
df_flights_6hours['end_lat'] = ''
df_flights_6hours['end_lon'] = ''
df_flights_6hours['start_lat'] = ''
df_flights_6hours['start_lon'] = ''

df_flights_24hours['end_lat'] = ''
df_flights_24hours['end_lon'] = ''
df_flights_24hours['start_lat'] = ''
df_flights_24hours['start_lon'] = ''

### Add values to the columns
for i in range(len(df_flights_6hours)):
    for j in range(len(df_airports)):
        if df_flights_6hours.iloc[i,1] == df_airports['IATA_CODE'][j]:
            df_flights_6hours['end_lat'][i] = df_airports['LATITUDE'][j]
            df_flights_6hours['end_lon'][i] = df_airports['LONGITUDE'][j]
        if df_flights_6hours.iloc[i,0] == df_airports['IATA_CODE'][j]:            
            df_flights_6hours['start_lat'][i] = df_airports['LATITUDE'][j]
            df_flights_6hours['start_lon'][i] = df_airports['LONGITUDE'][j]
            
for i in range(len(df_flights_24hours)):
    for j in range(len(df_airports)):
        if df_flights_24hours.iloc[i,1] == df_airports['IATA_CODE'][j]:
            df_flights_24hours['end_lat'][i] = df_airports['LATITUDE'][j]
            df_flights_24hours['end_lon'][i] = df_airports['LONGITUDE'][j]
        if df_flights_24hours.iloc[i,0] == df_airports['IATA_CODE'][j]:            
            df_flights_24hours['start_lat'][i] = df_airports['LATITUDE'][j]
            df_flights_24hours['start_lon'][i] = df_airports['LONGITUDE'][j]

C:\Users\hugoo\AppData\Local\Temp\ipykernel_5932\1005919128.py:23: FutureWarning:

Treating integers as positional in Series.__setitem__ with a Float64Index is deprecated. In a future version, `series[an_int] = val` will insert a new key into the Series. Use `series.iloc[an_int] = val` to treat the key as positional.

C:\Users\hugoo\AppData\Local\Temp\ipykernel_5932\1005919128.py:24: FutureWarning:

Treating integers as positional in Series.__setitem__ with a Float64Index is deprecated. In a future version, `series[an_int] = val` will insert a new key into the Series. Use `series.iloc[an_int] = val` to treat the key as positional.

C:\Users\hugoo\AppData\Local\Temp\ipykernel_5932\1005919128.py:26: FutureWarning:

Treating integers as positional in Series.__setitem__ with a Float64Index is deprecated. In a future version, `series[an_int] = val` will insert a new key into the Series. Use `series.iloc[an_int] = val` to treat the key as positional.

C:\Users\hugoo\AppData\Local\Temp\ipyker

In [9]:
# Exporting DataFrames to import in the main notebook            
df_airports.to_pickle("./airports")
df_flights_6hours.to_pickle("./flights_6hours")
df_flights_24hours.to_pickle("./flights_24hours")